# Pre-Processing
## Removing all the links, hashtags and emojis.

In [1]:
import regex as re
class StanceDetection:
    def __init__(self,test,train):
        self.testname=test
        self.trainname=train
    def html(self,sentence):
        html=-1
        dot=-1
        final=[]
        i=0
        while(i<len(sentence)):
            if(sentence[i][0].lower().find("http")!=-1):
                html=i
            if((re.search("co",sentence[i][0].lower()) or re.search("tco",sentence[i][0].lower())) and html!=-1):
                dot=i
            if(html!=-1 and dot!=-1 and dot<=len(sentence)-2):
                del sentence[html:dot+2]
                i=html
                html=-1
                dot=-1
            i+=1
        return sentence
    def process_file(self):
        for u,ele in enumerate([self.testname,self.trainname]):
            with open(ele) as file:
                data = file.read()
            linebyline=data.split('\n')
            sent_in=[]
            sent_out=[]
            dic={'ENG':0,'HIN':1,'O':2}
            map_={}
            i=0
            while(i<len(linebyline)):
                temp=linebyline[i].split()
                if(len(temp)>=2):
                    if(temp[0]=='meta' and len(temp)==3):
                        map_[temp[1]]=len(sent_in)
                        if(len(sent_in)!=0):
                            sent_in[-1]=self.html(sent_in[-1])
                        sent_in.append([])
                        sent_out.append(temp[2])
                    else:
                        if(temp[0].find("@")==len(temp[0])-1 or temp[0].find("#")==len(temp[0])-1):
                            i+=1
                        else:
                            data_clean=re.sub("[^a-zA-Z]+", "",temp[0])
                            if(len(data_clean)!=0):
                                sent_in[-1].append((data_clean,dic[temp[1].upper()]))
                i+=1
            sent_in[-1]=self.html(sent_in[-1])
            if(u==0):
                self.test_in,self.test_out,self.test_map=sent_in,sent_out,map_
            else:
                self.train_in,self.train_out,self.train_map=sent_in,sent_out,map_

In [2]:
obj=StanceDetection("test.txt","train.txt")
obj.process_file()
obj.train_in[obj.train_map['9809']]

[('hema', 1),
 ('I', 1),
 ('hate', 0),
 ('all', 0),
 ('type', 0),
 ('of', 0),
 ('parathes', 0),
 ('but', 0),
 ('I', 1),
 ('don', 1),
 ('t', 0),
 ('hate', 0),
 ('muli', 1),
 ('ke', 1),
 ('parathe', 1),
 ('Me', 0),
 ('farting', 0)]

# Embedding
## Embedding is retrieved using FLARE for each word and padded then according to generate a sentence embedding.

In [3]:
import numpy as np
import regex as re
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM, Bidirectional,Flatten,GRU,GlobalMaxPool1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import backend as K
from keras_self_attention import SeqSelfAttention
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
import torch
from torch.nn.utils.rnn import pad_sequence
ops.reset_default_graph()
# import tensorflow as tf
np_emb_train=np.load("FLARE_train.npy",allow_pickle=1)
np_emb_test=np.load("FLARE_test.npy",allow_pickle=1)

Using TensorFlow backend.


In [4]:
print(np_emb_train[0].shape)  
print(np_emb_test.shape)  
X_train = np_emb_train
X_test = np_emb_test
llist = []
for i in X_train:
      llist.append(torch.tensor(i[:31]))
X_train = pad_sequence(llist, batch_first=True)
llist = []
for i in X_test:
      llist.append(torch.tensor(i[:31]))
X_test = pad_sequence(llist, batch_first=True)

(18, 3072)
(1869,)


In [5]:
Y_train=np.load("train.npy")
Y_test=np.load("test.npy")

# MODEL
## We used a Bi-directional LSTM with 100 elements and then a LSTM with 20 element then densed it to 3 elements using a 3rd layer.

In [6]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def LSTM_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(128,input_shape=(31,3072),return_sequences=1)))
    model.add(Dropout(0.2))
    model.add(GRU(20))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc',f1_m,precision_m, recall_m])
    filepath="lstm.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(X_train.numpy(), Y_train, epochs=5, batch_size=256,callbacks=callbacks_list)
    return model
model=LSTM_model()

Train on 15131 samples
Epoch 1/5
15104/15131 [============================>.] - ETA: 0s - loss: 0.9716 - acc: 0.4997 - f1_m: 0.3221 - precision_m: 0.5913 - recall_m: 0.2352
Epoch 00001: loss improved from inf to 0.97141, saving model to lstm.hdf5
15131/15131 [==============================] - 98s 6ms/sample - loss: 0.9714 - acc: 0.4999 - f1_m: 0.3252 - precision_m: 0.5929 - recall_m: 0.2380
Epoch 2/5
15104/15131 [============================>.] - ETA: 0s - loss: 0.8701 - acc: 0.5766 - f1_m: 0.5103 - precision_m: 0.6326 - recall_m: 0.4291
Epoch 00002: loss improved from 0.97141 to 0.87011, saving model to lstm.hdf5
15131/15131 [==============================] - 95s 6ms/sample - loss: 0.8701 - acc: 0.5766 - f1_m: 0.5103 - precision_m: 0.6335 - recall_m: 0.4287
Epoch 3/5
15104/15131 [============================>.] - ETA: 0s - loss: 0.8347 - acc: 0.6082 - f1_m: 0.5568 - precision_m: 0.6625 - recall_m: 0.4814
Epoch 00003: loss improved from 0.87011 to 0.83482, saving model to lstm.hdf5
151

In [7]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test.numpy(), Y_test, verbose=0)

In [8]:
accuracy, f1_score, precision, recall

(0.52969503, 0.51582956, 0.54311556, 0.49189213)

In [9]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional multiple                  3277824   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
gru (GRU)                    multiple                  16680     
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  63        
Total params: 3,294,567
Trainable params: 3,294,567
Non-trainable params: 0
_________________________________________________________________
None
